In [15]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import copy
import folium
from pyeasyga import pyeasyga
import random
import copy


In [16]:
#CARGA DE DATOS
#Se reutiliza el código de carga de datos del proyecto anterior

def load_distance_time_dic(path):
    data = pd.read_csv(path)
    distance = {}
    time = {}
    for i in range(len(data)):
        origen = int(data.iloc[i, 0])
        destino = int(data.iloc[i, 1])
        distance[origen, destino] = float(data.iloc[i, 2])
        time[origen, destino] = float(data.iloc[i, 3])
    return distance, time
5
def load_vehicles(path):
    data = pd.read_csv(path)
    vehicles = {}
    for i in range(len(data)):
        id = int(data.iloc[i, 0])
        capacity = int(data.iloc[i, 1])
        ran = float(data.iloc[i, 2])
        vehicles[id] = (capacity, ran)
    return vehicles

def load_demand(path):
    data = pd.read_csv(path)
    demand_dic = {}
    for i in range(len(data)):
        id = int(data.iloc[i, 1])
        demand = float(data.iloc[i, 2])
        demand_dic[id] = demand
    return demand_dic

def load_coordinates(depotsPath, clientsPath):
    coord = {}
    depot = pd.read_csv(depotsPath)
    client = pd.read_csv(clientsPath)
    for i in range(len(depot)):
        id = int(depot.iloc[i, 1])
        lat = float(depot.iloc[i, 3])
        long = float(depot.iloc[i, 2])
        coord[id] = [lat, long]
    for j in range(len(client)):
        id = int(client.iloc[j, 1])
        lat = float(client.iloc[j, 4])
        long = float(client.iloc[j, 3])
        coord[id] = [lat, long]
    return coord


In [17]:
distancia,time_dic = load_distance_time_dic('../Datos/casoBase.csv')
vehiculos = load_vehicles('../Datos/vehicles.csv')
demanda = load_demand('../Datos/clients.csv')
coord = load_coordinates('../Datos/depots.csv', '../Datos/clients.csv')
print(f"Distancia: {distancia}")
print(f"Vehiculos: {vehiculos}")
print(f"Demanda: {demanda}")
print(f"Coordenadas: {coord}")
print(f"Time: {time_dic}")



Distancia: {(1, 2): 27144.3, (1, 3): 17677.1, (1, 4): 13979.6, (1, 5): 26650.8, (1, 6): 22583.7, (1, 7): 18562.5, (1, 8): 24765.9, (1, 9): 23929.4, (1, 10): 27835.9, (1, 11): 33651.8, (1, 12): 31029.4, (1, 13): 16124.7, (1, 14): 21612.2, (1, 15): 10387.4, (1, 16): 27040.2, (1, 17): 33963.1, (1, 18): 26541.3, (1, 19): 20053.2, (1, 20): 21278.2, (1, 21): 13411.2, (1, 22): 27869.5, (1, 23): 24262.6, (1, 24): 11364.8, (1, 25): 30809.8, (2, 1): 30809.9, (2, 3): 14256.3, (2, 4): 19400.3, (2, 5): 1130.9, (2, 6): 12475.1, (2, 7): 11416.7, (2, 8): 16851.4, (2, 9): 13820.8, (2, 10): 7509.8, (2, 11): 7110.4, (2, 12): 10890.2, (2, 13): 24445.1, (2, 14): 7973.9, (2, 15): 20142.5, (2, 16): 1255.4, (2, 17): 7421.8, (2, 18): 4283.4, (2, 19): 9819.6, (2, 20): 15281.3, (2, 21): 23046.5, (2, 22): 21147.7, (2, 23): 5482.0, (2, 24): 21239.2, (2, 25): 4880.9, (3, 1): 18222.2, (3, 2): 12672.2, (3, 4): 6812.6, (3, 5): 12178.7, (3, 6): 14735.1, (3, 7): 8279.5, (3, 8): 8286.5, (3, 9): 16080.8, (3, 10): 15410.7,

In [23]:
import random
from pyeasyga import pyeasyga
import matplotlib.pyplot as plt

# Parámetros globales
depot_id = 1
pf = 15000
ft = 5000
cm = 700
gv = 0.411458

# Función para evaluar rutas (igual que tu evaluar_individuo pero solo con rutas)
def evaluar_rutas(rutas):
    total_cost = 0
    for ruta in rutas:
        if len(ruta) <= 2:
            continue
        cap_max, rango_max = vehiculos[0]  # suponemos todos iguales, ajustar si quieres
        carga = 0
        distancia_total = 0
        for i in range(len(ruta)-1):
            a, b = ruta[i], ruta[i+1]
            distancia_total += distancia.get((a,b), 9999999)
            if b != depot_id:
                carga += demanda.get(b, 0)

        penalizacion = 0
        if carga > cap_max:
            penalizacion += 1e6 * (carga - cap_max)
        if distancia_total > rango_max:
            penalizacion += 1e6 * (distancia_total - rango_max)

        costo_vehiculo = gv*pf*distancia_total + ft + cm + penalizacion
        total_cost += costo_vehiculo
    return total_cost

# Función fitness para pyeasyga (nota que recibe un objeto Chromosome)
def fitness_function(individual, data):
    genes = individual.genes
    rutas = [[] for _ in vehiculos]
    for idx, c in enumerate(genes):
        rutas[idx % len(vehiculos)].append(c)
    rutas = [[depot_id] + r + [depot_id] if r else [] for r in rutas]
    return -evaluar_rutas(rutas)  # negativo para maximizar fitness en pyeasyga

# Generar datos iniciales (lista de clientes sin depósito)
def generar_datos(demanda, depot_id):
    return [c for c in demanda.keys() if c != depot_id]

# Algoritmo genético usando pyeasyga
def run_ga_pyeasyga(distancia, demanda, vehiculos, generaciones=200, poblacion=50):
    clientes = generar_datos(demanda, depot_id)

    ga = pyeasyga.GeneticAlgorithm(
        clientes,
        population_size=poblacion,
        generations=generaciones,
        crossover_probability=0.8,
        mutation_probability=0.2,
        elitism=True,
        maximise_fitness=True
    )

    ga.fitness_function = fitness_function

    # Inicializamos población con permutaciones aleatorias
    def create_individual(data):
        genes = data[:]
        random.shuffle(genes)
        return genes
    ga.create_individual = create_individual

    ga.create_first_generation()

    evolucion = []

    for gen in range(generaciones):
        # Evaluamos fitness y ordenamos
        fitness_scores = [(ga.fitness_function(ind, clientes), ind.genes) for ind in ga.current_generation]
        fitness_scores.sort(reverse=True, key=lambda x: x[0])
        evolucion.append(-fitness_scores[0][0])  # guardamos costo real positivo

        next_generation = []
        if ga.elitism:
            next_generation.append(fitness_scores[0][1])

        while len(next_generation) < poblacion:
            parent1 = random.choice(fitness_scores)[1]
            parent2 = random.choice(fitness_scores)[1]

            # Cruce simple (orden parcial)
            cut = len(parent1) // 2
            child1 = parent1[:cut] + [g for g in parent2 if g not in parent1[:cut]]
            child2 = parent2[:cut] + [g for g in parent1 if g not in parent2[:cut]]

            # Mutación simple: intercambio de dos genes
            def mutar(ind):
                ind = ind[:]
                if len(ind) > 1:
                    i1, i2 = random.sample(range(len(ind)), 2)
                    ind[i1], ind[i2] = ind[i2], ind[i1]
                return ind

            if random.random() < ga.mutation_probability:
                child1 = mutar(child1)
            if random.random() < ga.mutation_probability:
                child2 = mutar(child2)

            next_generation.append(child1)
            if len(next_generation) < poblacion:
                next_generation.append(child2)

        ga.current_generation = [pyeasyga.Chromosome(genes=ind) for ind in next_generation]

    # Mejor solución final
    fitness_scores = [(ga.fitness_function(ind, clientes), ind.genes) for ind in ga.current_generation]
    mejor = max(fitness_scores, key=lambda x: x[0])
    mejor_individuo = mejor[1]

    # Convertir mejor individuo a rutas
    rutas = [[] for _ in vehiculos]
    for idx, c in enumerate(mejor_individuo):
        rutas[idx % len(vehiculos)].append(c)
    rutas = [[depot_id] + r + [depot_id] if r else [] for r in rutas]

    print("Mejor costo encontrado:", -mejor[0])
    return rutas, -mejor[0], evolucion

# Función para graficar evolución
def graficar_evolucion_costos(evolucion):
    plt.figure(figsize=(10, 5))
    plt.plot(evolucion)
    plt.xlabel('Generación')
    plt.ylabel('Costo')
    plt.title('Evolución del costo en GA con pyeasyga')
    plt.grid(True)
    plt.show()


In [19]:
def visualizar_rutas_folium(rutas, coord):
    # Crear mapa centrado en depósito o promedio de coordenadas
    lat_dep, lon_dep = coord[depot_id]
    m = folium.Map(location=[lat_dep, lon_dep], zoom_start=11, tiles='Cartodb Positron')

    colors = ['blue', 'green', 'cyan', 'magenta','olive', 'blue', 'orange', 'purple','red']
    icons = ['blue', 'green', 'lightblue', 'pink','lightgreen', 'blue', 'orange', 'darkpurple','red']

    for v, ruta in enumerate(rutas):
        if not ruta or len(ruta) < 2:
            continue

        # Construir lista de coordenadas para PolyLine: [(lat, lon), (lat, lon), ...]
        coords_ruta = [coord[n] for n in ruta]

        # Dibujar la ruta
        folium.PolyLine(
            coords_ruta,
            color=colors[v % len(colors)],
            weight=5,
            opacity=0.7,
            tooltip=f'Vehículo {v}'
        ).add_to(m)

        # Marcador inicio (depósito)
        folium.Marker(
            coords_ruta[0],
            popup="Inicio (Depósito)",
            icon=folium.Icon(color='black', icon='home')
        ).add_to(m)

        # Marcador fin de ruta
        folium.Marker(
            coords_ruta[-1],
            popup=f"Llegada Vehículo {v}",
            icon=folium.Icon(color=icons[v % len(icons)], icon='flag')
        ).add_to(m)

    return m


In [24]:
rutas, mejor, evolucion = run_ga_pyeasyga(distancia, demanda, vehiculos, generaciones=200, poblacion=50)
def graficar_evolucion_costos(mejores_costos):
    plt.figure(figsize=(10, 5))
    plt.plot(mejores_costos, marker='o', color='green')
    plt.xlabel('Generación')
    plt.ylabel('Mejor costo')
    plt.title('Evolución del costo del mejor individuo')
    plt.grid(True)
    plt.show()

graficar_evolucion_costos(evolucion)


AttributeError: 'list' object has no attribute 'genes'